In [44]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np
import pandas as pd
import pycwt as wavelet
import scipy.constants as const
import xarray as xr
from cartopy.mpl.ticker import LatitudeFormatter, LongitudeFormatter
from matplotlib.pyplot import MultipleLocator
from pycwt.helpers import find
from pyproj import Transformer as geo_transformer
from scipy.signal import butter, detrend, filtfilt, savgol_filter

In [45]:
import gc
from pathlib import Path
from pprint import pprint

import tomllib

In [46]:
def lla2ecef(lon, lat, alt):
    lla = {'proj': 'latlong', 'ellps': 'WGS84', 'datum': 'WGS84'}
    ecef = {'proj': 'geocent', 'ellps': 'WGS84', 'datum': 'WGS84'}
    transformer = geo_transformer.from_crs(lla, ecef, always_xy=True)
    return transformer.transform(lon, lat, alt, radians=False)


def ecef2lla(x, y, z):
    lla = {'proj': 'latlong', 'ellps': 'WGS84', 'datum': 'WGS84'}
    ecef = {'proj': 'geocent', 'ellps': 'WGS84', 'datum': 'WGS84'}
    transformer = geo_transformer.from_crs(ecef, lla, always_xy=True)
    return transformer.transform(x, y, z, radians=False)


def format_func(x, pos=None):
    x = mdates.num2date(x)
    if x.hour == 0:
        fmt = '%H\n%b %d'
    else:
        fmt = '%H'
    label = x.strftime(fmt)

    return label


def measure_dist_bear_orient(epicenter_ll, IPP_ll, radians=False):
    """units: {distance: km, bearing: radians}"""

    #     radium = 6378+400
    # Height =  400*1000
    Height = 0

    lambda_a, phi_a = epicenter_ll
    if lambda_a < 0:
        lambda_a += 360

    lambda_b, phi_b = IPP_ll
    if lambda_b < 0:
        lambda_b += 360

    x1, y1, z1 = lla2ecef(lambda_a, phi_a, Height)
    x2, y2, z2 = lla2ecef(lambda_b, phi_b, Height)

    vector1 = np.array([x1, y1, z1])
    radium1 = np.sqrt(np.dot(vector1, vector1)) / 1000

    vector2 = np.array([x2, y2, z2])
    radium2 = np.sqrt(np.dot(vector2, vector2)) / 1000

    radium = (radium1 + radium2) / 2
    #     print('radium:', radium)

    if not radians:
        lambda_a = np.radians(lambda_a)
        lambda_b = np.radians(lambda_b)
        phi_a = np.radians(phi_a)
        phi_b = np.radians(phi_b)

    cosine = np.cos(phi_a) * np.cos(phi_b) * np.cos(lambda_a - lambda_b) + np.sin(
        phi_a
    ) * np.sin(phi_b)

    delta = np.arccos(cosine)
    distance = radium * delta

    tangent = (
        np.sin(lambda_b - lambda_a)
        * np.cos(phi_b)
        * np.cos(phi_a)
        / (np.sin(phi_b) - np.sin(phi_a) * cosine)
    )
    bearing = np.arctan2(abs(tangent), 1)

    if lambda_b > lambda_a:
        if phi_b > phi_a:
            bearing = bearing
        elif phi_b < phi_a:
            bearing = np.pi - bearing
        else:
            bearing = np.pi / 2

    elif lambda_b < lambda_a:
        if phi_b > phi_a:
            bearing = 2 * np.pi - bearing
        elif phi_b < phi_a:
            bearing = np.pi + bearing
        else:
            bearing = 3 * np.pi / 2

    else:
        if phi_b >= phi_a:
            bearing = 0
        elif phi_b < phi_a:
            bearing = np.pi

    quarter = np.pi / 4

    if bearing >= 0 and bearing < quarter:
        orientation = 'N.E.'

    elif bearing >= quarter and bearing < 2 * quarter:
        orientation = 'E.N.'

    elif bearing >= 2 * quarter and bearing < 3 * quarter:
        orientation = 'E.S.'

    elif bearing >= 3 * quarter and bearing < 4 * quarter:
        orientation = 'S.E.'

    elif bearing >= 4 * quarter and bearing < 5 * quarter:
        orientation = 'S.W.'

    elif bearing >= 5 * quarter and bearing < 6 * quarter:
        orientation = 'W.S.'

    elif bearing >= 6 * quarter and bearing < 7 * quarter:
        orientation = 'W.N.'

    else:
        orientation = 'N.W.'

    #     if np.degrees(bearing) >= 337.5 or np.degrees(bearing) < 22.5:
    #         orientation = 'N. '

    #     elif np.degrees(bearing) >= 22.5 and np.degrees(bearing) < 67.5:
    #         orientation = 'NE.'

    #     elif np.degrees(bearing) >= 67.5 and np.degrees(bearing) < 112.5:
    #         orientation = 'E. '

    #     elif np.degrees(bearing) >= 112.5 and np.degrees(bearing) < 157.5:
    #         orientation = 'SE.'

    #     elif np.degrees(bearing) >= 157.5 and np.degrees(bearing) < 202.5:
    #         orientation = 'S. '

    #     elif np.degrees(bearing) >= 202.5 and np.degrees(bearing) < 247.5:
    #         orientation = 'SW.'

    #     elif np.degrees(bearing) >= 247.5 and np.degrees(bearing) < 292.5:
    #         orientation = 'W. '

    #     elif np.degrees(bearing) >= 292.5 and np.degrees(bearing) < 337.5:
    #         orientation = 'NW.'

    #     else:
    #         orientation = 'W. '

    return distance, bearing, orientation

In [47]:
class Signal(object):
    def __init__(self, signals, sample_rate):
        self.name = 'signals'
        # self.signals = signals
        self.sample_rate = sample_rate

        nan_idx = np.isnan(signals)
        # 找到数组中非 NaN 值的位置
        not_nan_idx = np.logical_not(nan_idx)
        # 使用插值函数 interp() 对 NaN 值进行插值
        signals[nan_idx] = np.interp(
            nan_idx.nonzero()[0], not_nan_idx.nonzero()[0], signals[not_nan_idx]
        )
        self.signals = signals

    def butter_bandpass(self, cut_off_frequency_list, order=5):
        """带通滤波器，需要两个截止频率，以 list 形式输入"""
        self.nyq = 0.5 * self.sample_rate
        normal_cut_off = np.array(cut_off_frequency_list) / self.nyq
        b, a = butter(order, normal_cut_off, btype='bandpass', analog=False)
        return b, a

    def butter_bandpass_filtfilt(self, cut_off_frequency_list, order=3):
        """带通滤波器的执行，消除延迟"""
        b, a = self.butter_bandpass(cut_off_frequency_list, order=order)
        y = filtfilt(b, a, self.signals)
        return y

    def savgol_dtrend_filter(self, winlong, winshort, order=3):
        trend = savgol_filter(self.signals, winlong, order)
        if winshort == 0:
            dtrend = self.signals - trend
            return dtrend
        else:
            dtrend = self.signals - trend
            bandpass = savgol_filter(dtrend, winshort, order)
            return bandpass

In [48]:
def wavelet_transform(data):
    N = data.size
    data = Signal(data, 1)

    notrend = data.savgol_dtrend_filter(4 * 60 * 60 + 1, 0)
    std = np.std(notrend, ddof=1)
    norm = notrend / std
    variance = std**2

    ### wavelet setting
    # unit: 2min
    dt = 1.0 / 120
    s0 = 2  # Starting scale
    dj = 1.0 / 20  # 1/dj sub-octaves per octaves
    J = 6 / dj  # J*dj powers of two with dj sub-octaves

    wave_base = wavelet.Morlet(6)
    alpha, _, _ = wavelet.ar1(data.signals)  # Lag-1 autocorrelation for red noise

    ### wavelet transform
    wave, scales, freqs, coi, fft, fftfreqs = wavelet.cwt(
        norm, dt, dj, s0, J, wave_base
    )
    iwave = wavelet.icwt(wave, scales, dt, dj, wave_base) * std

    power = np.power(np.abs(wave), 2)
    fft_power = np.power(np.abs(fft), 2)

    period = 1 / freqs * 2
    fft_period = 1 / fftfreqs * 2
    coi *= 2
    power /= scales[:, None]  # calibration

    # Liu, Y., Liang, X. S. and Weisberg, R. H.
    # Rectification of the bias in the wavelet power spectrum.
    # Journal of Atmospheric and Oceanic Technology, 2007, 24, 2093-2102.
    # http://dx.doi.org/10.1175/2007JTECHO511.1

    signif, fft_theor = wavelet.significance(
        1.0, dt, scales, 0, alpha, significance_level=0.95, wavelet=wave_base
    )

    sig95 = np.ones([1, N]) * signif[:, None]
    sig95 = power / sig95

    glbl_power = power.mean(axis=1)
    dof = N - scales  # Correction for padding at edges
    glbl_signif, tmp = wavelet.significance(
        variance,
        dt,
        scales,
        1,
        alpha,
        significance_level=0.95,
        dof=dof,
        wavelet=wave_base,
    )

    return (
        notrend,
        period,
        power,
        coi,
        sig95,
        wave_base,
        glbl_signif,
        variance,
        fft_theor,
        glbl_power,
        iwave,
        fft_power,
        fft_period,
    )

In [49]:
def plot_wavelet_results(
    t_range,
    occur_UT,
    site,
    sv,
    dist,
    focus_posi,
    site_posi,
    ipp_posi,
    notrend,
    period,
    power,
    coi,
    sig95,
    wave_base,
    glbl_signif,
    variance,
    fft_theor,
    glbl_power,
    f_save,
    sat_posi=[0, 0, 0],
    # bear=None,
):
    title = 'DTEC Time Series'
    label = 'DTEC'
    units = 'TECU'

    x_start = np.datetime64('2023-12-18T00:00:00')
    x_end = np.datetime64('2023-12-19T00:00:00')
    # Prepare the figure
    fig = plt.figure(figsize=(11, 8))

    # First sub-plot, the original time series anomaly and inverse wavelet transform.
    ax = plt.axes([0.1, 0.75, 0.65, 0.2])
    ax.plot(t_range, notrend, 'k', linewidth=1.5)
    # ax.plot(
    #     t_range,
    #     iwave,
    #     '-',
    #     linewidth=1,
    #     # color=[0.5, 0.5, 0.5]
    #     color='cyan',
    # )
    ax.axvline(occur_UT, color='#cccccc', alpha=0.9, ls='--', lw=2)

    ax.set_title(f'a) {title}')
    ax.set_ylabel(f'{label} ({units})')
    ax.set_xlim([x_start, x_end])
    ticks = np.arange(0, 26, 2)
    ax.xaxis.set_major_locator(mdates.HourLocator(ticks))
    ax.xaxis.set_minor_locator(mdates.HourLocator())
    ax.xaxis.set_major_formatter(format_func)

    # Second sub-plot, the normalized wavelet power spectrum and significance
    res = '10m'
    proj = ccrs.PlateCarree()
    # minlon, maxlon, minlat, maxlat = 80, 128, 0, 48
    minlat, maxlat = 0, 46
    minlon = min(focus_posi[0], ipp_posi[0], sat_posi[sv][0])
    maxlon = max(focus_posi[0], ipp_posi[0], sat_posi[sv][0])

    if maxlon - minlon < 46:
        minlon = minlon - (46.0 - (maxlon - minlon)) / 2
        maxlon = maxlon + (46.0 - (maxlon - minlon)) / 2
    else:
        maxlon = minlon + 46

    extents = [minlon, maxlon, minlat, maxlat]
    bx = plt.axes([0.77, 0.75, 0.2, 0.2], projection=proj)
    bx.set_extent(extents, crs=proj)
    bx.add_feature(
        cfeature.LAND.with_scale(res), edgecolor='black', lw=0.12, facecolor='white'
    )
    bx.add_feature(cfeature.OCEAN.with_scale(res), facecolor='navy', alpha=0.06)
    bx.add_feature(cfeature.BORDERS.with_scale(res), edgecolor='black', lw=0.1)

    bx.scatter(
        focus_posi[0], focus_posi[1], c='crimson', transform=proj, s=30, marker='X'
    )
    bx.scatter(site_posi[0], site_posi[1], c='k', transform=proj, s=15, marker='^')
    bx.scatter(ipp_posi[0], ipp_posi[1], c='cyan', transform=proj, s=40, marker='o')
    bx.annotate(
        '',
        xy=(focus_posi[0], focus_posi[1]),
        xytext=(ipp_posi[0], ipp_posi[1]),
        arrowprops=dict(
            facecolor='powderblue',
            edgecolor='powderblue',
            width=0.2,
            headwidth=0.02,
            shrink=0.1,
        ),
    )

    # bx.annotate(
    #     f'{site}',
    #     xy=(ipp_posi[0], ipp_posi[1]),
    #     xytext=(2.5, -1.5),
    #     textcoords='offset points',
    # )
    bx.annotate(
        f'{dist:<5.0f} km',
        xy=((ipp_posi[0] + focus_posi[0]) / 2, (ipp_posi[1] + focus_posi[1]) / 2),
        xytext=(0.2, 2.5),
        textcoords='offset points',
    )
    bx.scatter(
        sat_posi[sv][0],
        sat_posi[sv][1] + 1.5,
        c='b',
        transform=proj,
        s=50,
        marker='^',
    )
    bx.annotate(
        f'{sv}',
        xy=(sat_posi[sv][0], sat_posi[sv][1]),
        xytext=(2.5, 0),
        textcoords='offset points',
    )
    bx.set_title(f'b) {site} IPP')
    # dx.set_xlabel(f'Longitude')
    # dx.set_ylabel(f'Latitude')
    bx.set_xticks(np.arange(int(minlon), maxlon + 10, 20), crs=proj)
    bx.set_yticks(np.arange(minlat, maxlat + 10, 15), crs=proj)
    bx.xaxis.set_major_formatter(LongitudeFormatter())
    bx.yaxis.set_major_formatter(LatitudeFormatter())

    # Third sub-plot, the global wavelet and Fourier power spectra and theoretical
    # level contour lines and cone of influece hatched area. Note that period
    # scale is logarithmic.
    cx = plt.axes([0.1, 0.37, 0.65, 0.28], sharex=ax)
    maxlev = (np.log2(power)).max()
    # maxlev = 7.1
    levels = np.arange(0, maxlev, 0.2)
    im = cx.contourf(
        t_range,
        period,
        np.log2(power),
        levels=levels,
        extend='both',
        cmap=plt.cm.jet,
    )
    cx.axvline(occur_UT, color='#cccccc', alpha=0.9, ls='--', lw=2)
    cx.contour(t_range, period, sig95, [-99, 1], colors='k', linewidths=2)
    cx.plot(t_range, coi, 'w--', lw=2.5)

    cx.set_title(f'c) {label} Wavelet Power Spectrum ({wave_base.name})')
    cx.set_xlabel('Time (UT)')
    cx.set_xlim([x_start, x_end])
    cx.set_ylabel('Period (minutes)')
    cx.set_ylim([4, 128])
    cx.set_yscale('log', base=2)
    cx.yaxis.set_major_formatter(ticker.ScalarFormatter())
    cx.ticklabel_format(axis='y', style='plain')
    cx.invert_yaxis()
    ticks = np.arange(0, 26, 2)
    # axs[-1,0].set_xlim(start, end)
    cx.xaxis.set_major_locator(mdates.HourLocator(ticks))
    cx.xaxis.set_minor_locator(mdates.HourLocator())
    cx.xaxis.set_major_formatter(format_func)

    position = fig.add_axes([0.2, 0.25, 0.45, 0.025])
    plt.colorbar(im, cax=position, orientation='horizontal', extend='both', pad=0.02)

    # # Fourth sub-plot, the scale averaged wavelet spectrum.
    # noise spectra. Note that period scale is logarithmic.
    dx = plt.axes([0.77, 0.37, 0.2, 0.28], sharey=cx)
    dx.plot(glbl_signif, period, 'k')
    # cx.plot(variance * fft_power, fft_period, '-', color='#cccccc',linewidth=1)
    dx.plot(variance * fft_theor, period, '--', color='#cccccc')
    dx.plot(variance * glbl_power, period, color='crimson', linewidth=2.5)
    dx.set_title('d) Global Wavelet Spectrum')
    dx.set_xlabel(f'Power ({units}$^2$)')
    dx.set_xlim([0, 20])
    dx.set_ylim([4, 128])
    dx.set_yscale('log', base=2)

    dx.yaxis.set_major_formatter(ticker.ScalarFormatter())
    dx.ticklabel_format(axis='y', style='plain')
    cx.invert_yaxis()
    plt.setp(dx.get_yticklabels(), visible=False)

    # plt.show()
    fig.savefig(f_save, bbox_inches='tight')
    # plt.show()
    plt.clf()
    plt.close('all')
    gc.collect()

In [50]:
p_toml = '/home/koi/pypoetry/earthquake_cases.toml'
event = 'Jishishan'

with open(p_toml, mode='rb') as f_toml:
    case = tomllib.load(f_toml)[event]
    pprint(case)

focus_posi, occur_UT = case['geo_posi'], case['occur_UT']
occur_UT = np.datetime64(occur_UT, 'm')

obs_net = 'CMONOC'
constellation = 'BeiDou'
H_orbit = 35786 * const.kilo
BeiDou_posi = {
    'C01': [140.0, 0, H_orbit],
    'C02': [80.07, 0, H_orbit],
    'C03': [110.45, 0, H_orbit],
    'C04': [159.98, 0, H_orbit],
    'C05': [58.71, 0, H_orbit],
}

t_interval = 1  # unit: second
f_nc = '/home/koi/pypoetry/BeiDou_TEC_352_01s_2023.nc'
nc = xr.open_dataset(f_nc)

{'depth': 10,
 'district': ['China', 'Gansu', 'Linxia', 'Jshishan'],
 'geo_posi': [102.79, 35.7],
 'magnitude': 6.2,
 'occur_UT': '2023-12-18T15:59:30'}


In [51]:
t_start = np.datetime64('2023-12-18T00:00:00')
t_end = np.datetime64('2023-12-19T00:00:00')
t_range = pd.date_range(t_start, t_end, freq=f'{t_interval}s')

# obs_sites = ['WUHN']
obs_sites = nc.observer.values
df = pd.read_csv('/home/koi/pypoetry/GNSS_TEC/CMONOC_sites.csv')

# site = 'WUHN'
sv = 'C01'
for site in obs_sites:
    svs = nc.sel(observer=site).sv.values
    if sv not in svs:
        continue
    else:
        f_save = f'/home/koi/pypoetry/GNSS_TEC/plots/wavelet_{site}_{sv}_TEC.png'

    # if Path(f_save).is_file():
    #     continue
    # else:
    #     pass

    site_posi = [
        df[df.OBSERVATION == site].LON.values,
        df[df.OBSERVATION == site].LAT.values,
    ]
    # pprint(site_posi)
    # continue
    # site_posi = [114.3573, 30.5317]
    stec = nc.sTEC_smth.sel(observer=site, sv=sv).dropna(dim='time')
    if stec.size < int(12 * 60 * 60 / t_interval):
        continue

    stec = stec.interp(
        time=t_range, method='linear', kwargs={'fill_value': 'extrapolate'}
    )
    zenith = nc.zenith.sel(observer=site, sv=sv).values
    ipp_posi = [
        nc.lon.sel(observer=site, sv=sv).values,
        nc.lat.sel(observer=site, sv=sv).values,
    ]

    dist, _, _ = measure_dist_bear_orient(focus_posi, ipp_posi)

    vtec = stec * np.cos(zenith)
    ### normalization
    data = vtec.values
    notrend = data.savgol_dtrend_filter(2 * 60 * 60 + 1, 0)
    std = notrend.std
    norm = notrend / std
    variance = std**2

    try:
        (
            notrend,
            period,
            power,
            coi,
            sig95,
            wave_base,
            glbl_signif,
            variance,
            fft_theor,
            glbl_power,
            _,
            _,
            _,
        ) = wavelet_transform(data)

        plot_wavelet_results(
            t_range,
            occur_UT,
            site,
            sv,
            dist,
            focus_posi,
            site_posi,
            ipp_posi,
            notrend,
            period,
            power,
            coi,
            sig95,
            wave_base,
            glbl_signif,
            variance,
            fft_theor,
            glbl_power,
            f_save,
            sat_posi=BeiDou_posi,
            # bear
        )
    except Exception as e:
        pprint(e)

/tmp/ipykernel_3918/1280536462.py:187: UserWarning: Attempt to set non-positive ylim on a log-scaled axis will be ignored.
  plt.clf()
/tmp/ipykernel_3918/1280536462.py:187: UserWarning: Attempt to set non-positive ylim on a log-scaled axis will be ignored.
  plt.clf()


Warning('Cannot place an upperbound on the unbiased AR(1). Series is too short or trend is to large.')


/tmp/ipykernel_3918/1280536462.py:187: UserWarning: Attempt to set non-positive ylim on a log-scaled axis will be ignored.
  plt.clf()
/tmp/ipykernel_3918/1280536462.py:187: UserWarning: Attempt to set non-positive ylim on a log-scaled axis will be ignored.
  plt.clf()
/tmp/ipykernel_3918/1280536462.py:187: UserWarning: Attempt to set non-positive ylim on a log-scaled axis will be ignored.
  plt.clf()
/tmp/ipykernel_3918/1280536462.py:187: UserWarning: Attempt to set non-positive ylim on a log-scaled axis will be ignored.
  plt.clf()
/tmp/ipykernel_3918/1280536462.py:187: UserWarning: Attempt to set non-positive ylim on a log-scaled axis will be ignored.
  plt.clf()
/tmp/ipykernel_3918/1280536462.py:187: UserWarning: Attempt to set non-positive ylim on a log-scaled axis will be ignored.
  plt.clf()
/tmp/ipykernel_3918/1280536462.py:187: UserWarning: Attempt to set non-positive ylim on a log-scaled axis will be ignored.
  plt.clf()
/tmp/ipykernel_3918/1280536462.py:187: UserWarning: Att

Warning('Cannot place an upperbound on the unbiased AR(1). Series is too short or trend is to large.')


/tmp/ipykernel_3918/1280536462.py:187: UserWarning: Attempt to set non-positive ylim on a log-scaled axis will be ignored.
  plt.clf()
/tmp/ipykernel_3918/1280536462.py:187: UserWarning: Attempt to set non-positive ylim on a log-scaled axis will be ignored.
  plt.clf()
/tmp/ipykernel_3918/1280536462.py:187: UserWarning: Attempt to set non-positive ylim on a log-scaled axis will be ignored.
  plt.clf()
/tmp/ipykernel_3918/1280536462.py:187: UserWarning: Attempt to set non-positive ylim on a log-scaled axis will be ignored.
  plt.clf()
/tmp/ipykernel_3918/1280536462.py:187: UserWarning: Attempt to set non-positive ylim on a log-scaled axis will be ignored.
  plt.clf()
/tmp/ipykernel_3918/1280536462.py:187: UserWarning: Attempt to set non-positive ylim on a log-scaled axis will be ignored.
  plt.clf()


...

Warning('Cannot place an upperbound on the unbiased AR(1). Series is too short or trend is to large.')
Warning('Cannot place an upperbound on the unbiased AR(1). Series is too short or trend is to large.')
Warning('Cannot place an upperbound on the unbiased AR(1). Series is too short or trend is to large.')
Warning('Cannot place an upperbound on the unbiased AR(1). Series is too short or trend is to large.')


/tmp/ipykernel_3918/1280536462.py:187: UserWarning: Attempt to set non-positive ylim on a log-scaled axis will be ignored.
  plt.clf()
/tmp/ipykernel_3918/1280536462.py:187: UserWarning: Attempt to set non-positive ylim on a log-scaled axis will be ignored.
  plt.clf()
/tmp/ipykernel_3918/1280536462.py:187: UserWarning: Attempt to set non-positive ylim on a log-scaled axis will be ignored.
  plt.clf()
/tmp/ipykernel_3918/1280536462.py:187: UserWarning: Attempt to set non-positive ylim on a log-scaled axis will be ignored.
  plt.clf()
/tmp/ipykernel_3918/1280536462.py:187: UserWarning: Attempt to set non-positive ylim on a log-scaled axis will be ignored.
  plt.clf()
/tmp/ipykernel_3918/1280536462.py:187: UserWarning: Attempt to set non-positive ylim on a log-scaled axis will be ignored.
  plt.clf()
/tmp/ipykernel_3918/1280536462.py:187: UserWarning: Attempt to set non-positive ylim on a log-scaled axis will be ignored.
  plt.clf()
/tmp/ipykernel_3918/1280536462.py:187: UserWarning: Att

Warning('Cannot place an upperbound on the unbiased AR(1). Series is too short or trend is to large.')


/tmp/ipykernel_3918/1280536462.py:187: UserWarning: Attempt to set non-positive ylim on a log-scaled axis will be ignored.
  plt.clf()
/tmp/ipykernel_3918/1280536462.py:187: UserWarning: Attempt to set non-positive ylim on a log-scaled axis will be ignored.
  plt.clf()
/tmp/ipykernel_3918/1280536462.py:187: UserWarning: Attempt to set non-positive ylim on a log-scaled axis will be ignored.
  plt.clf()
/tmp/ipykernel_3918/1280536462.py:187: UserWarning: Attempt to set non-positive ylim on a log-scaled axis will be ignored.
  plt.clf()
/tmp/ipykernel_3918/1280536462.py:187: UserWarning: Attempt to set non-positive ylim on a log-scaled axis will be ignored.
  plt.clf()
/tmp/ipykernel_3918/1280536462.py:187: UserWarning: Attempt to set non-positive ylim on a log-scaled axis will be ignored.
  plt.clf()
/tmp/ipykernel_3918/1280536462.py:187: UserWarning: Attempt to set non-positive ylim on a log-scaled axis will be ignored.
  plt.clf()
/tmp/ipykernel_3918/1280536462.py:187: UserWarning: Att

Warning('Cannot place an upperbound on the unbiased AR(1). Series is too short or trend is to large.')
